<a href="https://colab.research.google.com/github/JakeOh/202007_itw_bd18/blob/master/lab_python/python57_groupby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tips = sns.load_dataset('tips')

In [3]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


성별 팁의 평균

In [7]:
# 성별이 남성인 부분집합에서 팁 컬럼의 평균을 계산.
tip_male = tips[tips['sex'] == 'Male']['tip'].mean()
tip_male

3.0896178343949052

In [8]:
tip_female = tips[tips['sex'] == 'Female']['tip'].mean()
tip_female

2.833448275862069

In [9]:
pd.Series(data=[tip_male, tip_female], index=['Male', 'Female'])

Male      3.089618
Female    2.833448
dtype: float64

In [18]:
genders = tips['sex'].unique()  #> ['Female', 'Male']
tip_by_gender = []  # 성별 팁의 평균들을 저장할 리스트
for gender in genders:
    tip_mean = tips[tips['sex'] == gender]['tip'].mean()
    tip_by_gender.append(tip_mean)

s = pd.Series(data=tip_by_gender, index=genders)
s

Female    2.833448
Male      3.089618
dtype: float64

In [27]:
def group_by_mean(df, by, value):
    """데이터 프레임 df에서 컬럼(by)별 변수(value)의 평균을 계산해서 Series를 리턴"""
    categories = df[by].unique()
    value_by_cat = []
    for cat in categories:
        subset = df[df[by] == cat]  # split(데이터 프레임을 부분집합으로 "분리")
        mean = subset[value].mean()  # apply(각각의 부분집합에 통계 함수를 "적용")
        value_by_cat.append(mean)

    s = pd.Series(data=value_by_cat, index=categories)  # combine(결과를 하나로 "합침")
    
    return s

In [28]:
group_by_mean(df=tips, by='sex', value='tip')

Female    2.833448
Male      3.089618
dtype: float64

In [29]:
group_by_mean(df=tips, by='day', value='tip')

Sun     3.255132
Sat     2.993103
Thur    2.771452
Fri     2.734737
dtype: float64

In [30]:
group_by_mean(df=tips, by='day', value='total_bill')

Sun     21.410000
Sat     20.441379
Thur    17.682742
Fri     17.151579
dtype: float64

성별 팁의 평균: pivot_table 함수 사용

In [31]:
tips.pivot_table(values='tip', index='sex', aggfunc=np.mean)
# pd.DataFrame을 리턴!

,tip
sex,
Male,3.089618
Female,2.833448


성별 팁의 평균: groupby 함수 사용

In [32]:
grouped = tips.groupby(by='sex')  # splitting
grouped

In [34]:
for x in grouped:
    print(x)  #> (by, subset)
    print('*' * 50, '\n')

('Male',      total_bill   tip   sex smoker  day    time  size
1         10.34  1.66  Male     No  Sun  Dinner     3
2         21.01  3.50  Male     No  Sun  Dinner     3
3         23.68  3.31  Male     No  Sun  Dinner     2
5         25.29  4.71  Male     No  Sun  Dinner     4
6          8.77  2.00  Male     No  Sun  Dinner     2
..          ...   ...   ...    ...  ...     ...   ...
236       12.60  1.00  Male    Yes  Sat  Dinner     2
237       32.83  1.17  Male    Yes  Sat  Dinner     2
239       29.03  5.92  Male     No  Sat  Dinner     3
241       22.67  2.00  Male    Yes  Sat  Dinner     2
242       17.82  1.75  Male     No  Sat  Dinner     2

[157 rows x 7 columns])
************************************************** 

('Female',      total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
11        35.26  5.00  Female     No   Sun  Dinner     4
14        14.83  3.02  Fem

In [35]:
for by, subset in grouped:
    print('category:', by)
    print(subset)
    print('*' * 50, '\n')

category: Male
     total_bill   tip   sex smoker  day    time  size
1         10.34  1.66  Male     No  Sun  Dinner     3
2         21.01  3.50  Male     No  Sun  Dinner     3
3         23.68  3.31  Male     No  Sun  Dinner     2
5         25.29  4.71  Male     No  Sun  Dinner     4
6          8.77  2.00  Male     No  Sun  Dinner     2
..          ...   ...   ...    ...  ...     ...   ...
236       12.60  1.00  Male    Yes  Sat  Dinner     2
237       32.83  1.17  Male    Yes  Sat  Dinner     2
239       29.03  5.92  Male     No  Sat  Dinner     3
241       22.67  2.00  Male    Yes  Sat  Dinner     2
242       17.82  1.75  Male     No  Sat  Dinner     2

[157 rows x 7 columns]
************************************************** 

category: Female
     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
11        35.26  5.00  Female     No   Sun  Dinner     4
14        14.83

In [36]:
# apply-combine
grouped['tip'].mean()  # pandas.Series 리턴.

sex
Male      3.089618
Female    2.833448
Name: tip, dtype: float64

In [37]:
tips.groupby('sex')['tip'].mean()

sex
Male      3.089618
Female    2.833448
Name: tip, dtype: float64

In [38]:
# 성별 영수증금액, 팁의 평균
tips.groupby('sex')[['total_bill', 'tip']].mean()

,total_bill,tip
sex,,
Male,20.744076,3.089618
Female,18.056897,2.833448


In [39]:
tips.pivot_table(values=['total_bill', 'tip'], index='sex', aggfunc=np.mean)

,tip,total_bill
sex,,
Male,3.089618,20.744076
Female,2.833448,18.056897
